## Import Libraries 

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import statistics
import random
%matplotlib inline
plt.style.use('bmh')

In [2]:
from keras.preprocessing.image import ImageDataGenerator

2023-04-26 19:14:43.927077: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from skimage import exposure
from skimage.feature import local_binary_pattern
from skimage.util import img_as_ubyte
from skimage.feature import graycomatrix, graycoprops

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [5]:
import h5py

## Load data

In [6]:
h5f = h5py.File('data_h5','r')

denim_blended = h5f['denim_blended'][:]
denim = h5f['denim'][:]

knit_blended = h5f['knit_blended'][:]
knit = h5f['knit'][:]

fleece_blended = h5f['fleece_blended'][:]
fleece_poly = h5f['fleece_poly'][:]
fleece_Modal = h5f['fleece_Modal'][:]

linen = h5f['linen'][:]
linen_rayon = h5f['linen_rayon'][:]
linen_viscose = h5f['linen_viscose'][:]

leather_poly = h5f['leather_poly'][:]
leather_PU = h5f['leather_PU'][:]
leather_PVC = h5f['leather_PVC'][:]

outdoor_cotton = h5f['outdoor_cotton'][:]
outdoor_poly = h5f['outdoor_poly'][:]

# Preprocess Data 

### 1. Augment data

In [7]:
#should be np array (#imgs, pixel,pixel,1) for grey scale
def rotate_imgs (fabric, num_imgs, angle_start, angle_stop): 
    
    pixels = len(fabric[1,:,:,0]) 
    fabric_aug = np.zeros((num_imgs,pixels,pixels,3))
    
    for i in range(num_imgs): 
        angle = random.randint(angle_start, angle_stop)
        # ImageDataGenerator rotation
        datagen = ImageDataGenerator(rotation_range=angle, fill_mode='nearest')
        # iterator
        aug_iter = datagen.flow(fabric, batch_size=1)
        # generate one img with random angle
        image = next(aug_iter)[0].astype('uint8')
        fabric_aug[i,:,:,:] = image
 
    return fabric_aug

In [8]:
#should be np array (#imgs, pixel,pixel,1) for grey scale
def shift_imgs (fabric, num_imgs, width_shift, height_shift): 
    
    pixels = len(fabric[1,:,:,0]) 
    fabric_aug = np.zeros((num_imgs,pixels,pixels,3))
    
    for i in range(num_imgs): 
        width_shift = random.uniform(0.01, width_shift)
        height_shift = random.uniform(0.01, height_shift)
        # ImageDataGenerator rotation
        datagen = ImageDataGenerator(width_shift_range=width_shift, height_shift_range=height_shift)
        # iterator
        aug_iter = datagen.flow(fabric, batch_size=1)
        # generate one img with random angle
        image = next(aug_iter)[0].astype('uint8')
        fabric_aug[i,:,:,:] = image
 
    return fabric_aug

In [9]:
#should be np array (#imgs, pixel,pixel,1) for grey scale
def flip_imgs (fabric, num_imgs): 
    
    pixels = len(fabric[1,:,:,0]) 
    fabric_aug = np.zeros((num_imgs,pixels,pixels,3))
    
    for i in range(num_imgs): 
        # ImageDataGenerator rotation
        datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)
        # iterator
        aug_iter = datagen.flow(fabric, batch_size=1)
        # generate one img with random angle
        image = next(aug_iter)[0].astype('uint8')
        fabric_aug[i,:,:,:] = image
 
    return fabric_aug

In [10]:
#should be np array (#imgs, pixel,pixel,1) for grey scale
def zoom_imgs (fabric, num_imgs, start_range, stop_range): 
    
    pixels = len(fabric[1,:,:,0]) 
    fabric_aug = np.zeros((num_imgs,pixels,pixels,3))
    
    for i in range(num_imgs): 
        zoom = random.uniform(start_range,stop_range)
        #stop = random.uniform(start_range+.01, stop_range)
        # ImageDataGenerator rotation
        datagen = ImageDataGenerator(zoom_range=zoom)
        # iterator
        aug_iter = datagen.flow(fabric, batch_size=1)
        # generate one img with random angle
        image = next(aug_iter)[0].astype('uint8')
        fabric_aug[i,:,:,:] = image
 
    return fabric_aug

In [11]:
def transform_fabric(fabric):
    
    #num_imgs = len(fabric)
    num_imgs = 5
    pixels = len(fabric[1,:,:,0])
    
    rotate = rotate_imgs(fabric, num_imgs, -15, 15)
    shift = shift_imgs(fabric, num_imgs, 0.2, 0.2)
    flip = flip_imgs(fabric, num_imgs)
    zoom = zoom_imgs(fabric, num_imgs, 0.05, 1)
    
    transform = np.zeros((4*num_imgs,pixels,pixels,3))
    i = 0
    j = 0
    while j<(4*num_imgs):
        transform[j,:,:] = rotate[i]
        transform[j+1,:,:] = shift[i]
        transform[j+2,:,:] = flip[i]
        transform[j+3,:,:] = zoom[i]
        j+=4
        i+=1
       
    return transform


### Add augmented data 

In [12]:
denim_blended_aug = transform_fabric(denim_blended)
denim_blended = np.vstack((denim_blended_aug, denim_blended))
denim_aug = transform_fabric(denim)
denim = np.vstack((denim, denim_aug))

knit_blended_aug = transform_fabric(knit_blended)
knit_blended = np.vstack((knit_blended_aug, knit_blended))
knit_aug = transform_fabric(knit)
knit = np.vstack((knit, knit_aug))

fleece_blended_aug = transform_fabric(fleece_blended)
fleece_blended = np.vstack((fleece_blended_aug, fleece_blended))
fleece_poly_aug = transform_fabric(fleece_poly)
fleece_poly = np.vstack((fleece_poly, fleece_poly_aug))
fleece_Modal_aug = transform_fabric(fleece_Modal)
fleece_Modal = np.vstack((fleece_Modal, fleece_Modal_aug))


linen_aug = transform_fabric(linen)
linen_blended = np.vstack((linen_aug, linen))
linen_rayon_aug = transform_fabric(linen_rayon)
linen_rayon = np.vstack((linen_rayon, linen_rayon_aug))
linen_viscose_aug = transform_fabric(linen_viscose)
linen_viscose = np.vstack((linen_viscose, linen_viscose_aug))

leather_poly_aug = transform_fabric(leather_poly)
leather_poly = np.vstack((leather_poly_aug, leather_poly))
leather_PU_aug = transform_fabric(leather_PU)
leather_PU = np.vstack((leather_PU, leather_PU_aug))
leather_PVC_aug = transform_fabric(leather_PVC)
leather_PVC = np.vstack((leather_PVC, leather_PVC_aug))

outdoor_cotton_aug = transform_fabric(outdoor_cotton)
outdoor_cotton = np.vstack((outdoor_cotton, outdoor_cotton_aug))
outdoor_poly_aug = transform_fabric(outdoor_poly)
outdoor_poly = np.vstack((outdoor_poly, outdoor_poly_aug))

### Enhance images with histogram equlaization 

In [13]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

In [14]:
def create_gray_imgs(fabric):
    num_imgs = len(fabric)
    gray = np.zeros((num_imgs, 224, 224))
    for i in range(num_imgs):
        gray[i,:,:] = rgb2gray(fabric[i,:,:,:])
    return gray 

In [15]:
def equalize_img(image): 
    image = rgb2gray(image)
    image = image/255
    img_adapteq = exposure.equalize_adapthist(image, clip_limit=0.03) #tried clip_limit=[0.1-1.0] (0.5 was best) too grainy if you go up very soft going down 
    
    return img_adapteq

In [16]:
def equalize_fabric(fabric): 
    N,p,p,c = fabric.shape 
    equalized = np.zeros((N,p,p))
    for n in range(N):
        image = fabric[n,:,:,:]
        equalized[n,:,:] = equalize_img(image)
        
    return equalized

In [17]:
denim_eq = equalize_fabric(denim)
denim_b_eq = equalize_fabric(denim_blended)


knit_eq = equalize_fabric(knit)
knit_b_eq = equalize_fabric(knit_blended)


fleece_blended_eq = equalize_fabric(fleece_blended)
fleece_poly_eq = equalize_fabric(fleece_poly)
fleece_Modal_eq = equalize_fabric(fleece_Modal)


linen_eq = equalize_fabric(linen)
linen_rayon_eq = equalize_fabric(linen_rayon)
linen_viscose_eq = equalize_fabric(linen_viscose)


leather_poly_eq = equalize_fabric(leather_poly)
leather_PU_eq = equalize_fabric(leather_PU)
leather_PVC_eq = equalize_fabric(leather_PVC)


outdoor_cotton_eq = equalize_fabric(outdoor_cotton)
outdoor_poly_eq = equalize_fabric(outdoor_poly)

## Combine data 


In [18]:
def create_labels(fabric, label):
    images = len(fabric)
    labels = np.zeros((images))
    for i in range(images):
        labels[i] = label
        
    return labels

In [19]:
def create_data_matrix_eq(num_fabrics,fabric1,fabric2,fabric3,fabric4): 
    
    # create labels 
    fabric1_labels = create_labels(fabric1,0)
    fabric2_labels = create_labels(fabric2,1)
    
    if num_fabrics ==2: 
        X = np.vstack((fabric1, fabric2))
        Y = np.hstack((fabric1_labels, fabric2_labels))
    
    elif num_fabrics == 3: 
        fabric3_labels = create_labels(fabric3,2)
        X = np.vstack((fabric1, fabric2, fabric3))
        Y = np.hstack((fabric1_labels, fabric2_labels, fabric3_labels))
  
    return X, Y 
    

In [20]:
def create_data_matrix(num_fabrics,fabric1,fabric2,fabric3,fabric4): 
    
    # create gray images 
    fabric1_g = create_gray_imgs(fabric1)
    fabric2_g = create_gray_imgs(fabric2)
    
    # create labels 
    fabric1_labels = create_labels(fabric1_g,0)
    fabric2_labels = create_labels(fabric2_g,1)
    
    row = len(fabric1_g)
    row2 = len(fabric1_labels)
    
    row3 = len(fabric2_g) 
    row4 = len(fabric2_labels)
    
    if num_fabrics == 2: 
        if row != row2: 
            print('error fabric1')
        elif row3 != row4: 
            print ('error fabric 2')
        else:
            X = np.vstack((fabric1_g, fabric2_g))
            Y = np.hstack((fabric1_labels, fabric2_labels))
    
    elif num_fabrics == 3: 
        fabric3_g = create_gray_imgs(fabric3)
        fabric3_labels = create_labels(fabric3_g,2)
        X = np.vstack((fabric1_g, fabric2_g, fabric3_g))
        Y = np.hstack((fabric1_labels, fabric2_labels, fabric3_labels))
        
    elif num_fabrics == 4: 
        fabric3_g = create_gray_imgs(fabric3)
        fabric3_labels = create_labels(fabric3_g,2)
        fabric4_g = create_gray_imgs(fabric4)
        fabric4_labels = create_labels(fabric4_g,3)
        X = np.vstack((fabric1_g, fabric2_g, fabric3_g,fabric4_g))
        Y = np.hstack((fabric1_labels, fabric2_labels, fabric3_labels, fabric4_labels))
        
    
    return X, Y 
    

In [21]:
X_fleece, Y_fleece = create_data_matrix_eq(3,fleece_blended_eq,fleece_poly_eq[0:90,:,:],fleece_Modal_eq,0)
X_knit, Y_knit = create_data_matrix_eq(2,knit_b_eq, knit_eq, 0, 0)
X_linen, Y_linen = create_data_matrix_eq(3,linen_eq,linen_rayon_eq, linen_viscose_eq,0)
X_leather, Y_leather = create_data_matrix_eq(3,leather_poly_eq,leather_PU_eq, leather_PVC_eq, 0)
X_outdoor, Y_outdoor = create_data_matrix_eq(2,outdoor_cotton_eq,outdoor_poly_eq, 0,0)
X_denim, Y_denim = create_data_matrix_eq(2,denim_eq,denim_b_eq, 0,0)

# Feature Extraction

### 1. LBP histograms  

In [22]:
def lbp_image(images,num_points,radius):
    lbps = []
    N, p, p = images.shape
    for n in range(N):
        lbp = local_binary_pattern(images[n,:,:], P=num_points, R=radius)
        lbps.append(lbp)
    
    return np.array(lbps)

In [23]:
## add note about where code is from 

def show_images_with_labels(images, labels, start_index=0):
    fig, axes = plt.subplots(ncols=7, nrows=1, figsize=(18, 2.5))

    index = start_index
    for i in range(7):
        axes[i].imshow(images[index], cmap='gray')
        axes[i].set_title([labels[index]])
        axes[i].get_xaxis().set_visible(False)
        axes[i].get_yaxis().set_visible(False)
        index += 1
    plt.show()

In [24]:
def create_sub_images(image, num_sub_imgs): 
    pixel, pixel = image.shape
    total_imgs = round(num_sub_imgs**2)
    sub_pixels = round(pixel/num_sub_imgs)
    #for n in range(N): 
    start_row = 0
    stop_row = start_row + sub_pixels
    start_col = 0
    stop_col = start_col +  sub_pixels
    row = 0 
    sub_imgs = np.zeros((total_imgs,sub_pixels,sub_pixels))
    for i in range(total_imgs-1):
        width = pixel/num_sub_imgs
        sub_imgs[i,:,:] = image[start_row:stop_row,start_col:stop_col]
        #change row 
        if stop_col ==224: 
            row = row+1 
            start_col = 0
            stop_col = start_col +  sub_pixels
            start_row = row*(sub_pixels)
            stop_row = start_row + sub_pixels 
        else: 
            start_col = start_col+sub_pixels
            stop_col= stop_col + sub_pixels
            
    return sub_imgs

In [25]:
def create_histograms(fabric,num_sub_imgs,bins_per_sub_images): 
    N,p,p = fabric.shape
    fabric_hist = np.zeros((N,round(bins_per_sub_images*num_sub_imgs*num_sub_imgs)))
    for n in range(N):
        image = fabric[n,:,:]
        sub_imgs = create_sub_images(image, num_sub_imgs)
        sub_img_hist = []
        for i in range(round(num_sub_imgs**2)): 
            histogram = np.histogram(sub_imgs[i,:,:], bins=bins_per_sub_images)[0]
            sub_img_hist.append(histogram)
            
        hist1 = np.array(sub_img_hist).flatten()
        fabric_hist[n,:] = hist1
    
    return fabric_hist

In [26]:
def lbp_histograms(fabric, num_points, radius, num_sub_imgs, num_bins):
    lbp = lbp_image(fabric,num_points,radius)
    lbp_hist = create_histograms(lbp, num_sub_imgs, num_bins)
    
    return lbp_hist

### 2. GLCM

In [27]:
def GLCM_sub_imgs(sub_images, dist, angle): 
    sub_images = img_as_ubyte(sub_images/255)
    num_images, pixel, pixel = sub_images.shape
    contrast = []
    dissimilarity = []
    homogeneity = []
    ASM = []
    energy = []
    correlation = []
    for i in range(num_images): 
        glcm = graycomatrix(sub_images[i,:,:], distances = dist, angles = angle, levels = 256,
                       symmetric = True, normed = True)
        contrast.append(graycoprops(glcm, 'contrast'))
        dissimilarity.append(graycoprops(glcm, 'dissimilarity'))
        homogeneity.append(graycoprops(glcm, 'homogeneity'))
        ASM.append(graycoprops(glcm, 'ASM'))
        energy.append(graycoprops(glcm, 'energy'))
        correlation.append(graycoprops(glcm, 'correlation'))
    
    return contrast, dissimilarity, homogeneity, ASM, energy, correlation

In [28]:
def GLCM_fabric(fabric, num_sub_imgs, dist, angle):
    #fabric_g = create_gray_imgs(fabric)
    N, pixels, pixels = fabric.shape
    properties = np.zeros((N,6,round(num_sub_imgs**2)))
    for n in range(N): 
        image = fabric[n, :, :]
        sub_imgs = create_sub_images(image, num_sub_imgs)
        contrast, dissimilarity, homogeneity, ASM, energy, correlation = GLCM_sub_imgs(sub_imgs,dist,angle)
        for i in range(round(num_sub_imgs**2)): 
            properties[n,0,i]=contrast[i]
            properties[n,1,i]=dissimilarity[i]
            properties[n,2,i]=homogeneity[i]
            properties[n,3,i]= ASM[i]
            properties[n,4,i]= energy[i]
            properties[n,5,i]=correlation[i]
    
    return properties

## 3. combine features 

In [29]:
def fabric_features(X, Y, numpoints, radius): 
    
    #split data 
    X_train, X_test, y_train, y_test = train_test_split(X, Y, 
                                                    test_size=0.2,
                                                    random_state=42,
                                                       stratify = Y)
    
    #create lbp histograms 
    X_train_lbp = lbp_histograms(X_train, numpoints, radius, num_sub_imgs = 4, num_bins = 64) #tried 10 bins, 40 bins, 56 and 64 (drastically increased) 
    X_test_lbp = lbp_histograms(X_test, numpoints, radius, num_sub_imgs = 4, num_bins = 64)
    
    #create GCLM features 
    X_train_prop = GLCM_fabric(X_train, num_sub_imgs = 4, dist = [2], angle = [0])
    N,K,L = X_train_prop.shape
    X_train_prop = X_train_prop.reshape((N,K*L))
    X_test_prop = GLCM_fabric(X_test, num_sub_imgs = 4, dist = [2] , angle = [0])
    N,K,L = X_test_prop.shape
    X_test_prop = X_test_prop.reshape((N,K*L))
    
    #combine features 
    X_train = np.hstack((X_train_lbp,X_train_prop))
    X_test = np.hstack((X_test_lbp,X_test_prop))
    
    return X_train, X_test, y_train, y_test 

In [30]:
#fabric_features(X, Y, numpoints, radius)
X_train_fleece, X_test_fleece, Y_train_fleece, Y_test_fleece = fabric_features(X_fleece, Y_fleece,8,1)
X_train_knit, X_test_knit, Y_train_knit, Y_test_knit = fabric_features(X_knit, Y_knit,8,1)
X_train_linen, X_test_linen, Y_train_linen, Y_test_linen = fabric_features(X_linen, Y_linen,8,1)
X_train_leather, X_test_leather, Y_train_leather, Y_test_leather = fabric_features(X_leather, Y_leather,8,1)
X_train_outdoor, X_test_outdoor, Y_train_outdoor, Y_test_outdoor = fabric_features(X_outdoor, Y_outdoor,8,1)
X_train_denim, X_test_denim, Y_train_denim, Y_test_denim = fabric_features(X_denim, Y_denim,8,1)

In [31]:
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import rand_score
from sklearn.mixture import GaussianMixture

In [32]:
def kmeans_fabric(X_train, X_test, Y_test, num_clusters):
    kmeans = KMeans(n_clusters = num_clusters).fit(X_train)
    labels = kmeans.predict(X_test)
    rand = rand_score(Y_test, labels)
    
    return rand 

def GMM_fabric(X_train, X_test, Y_test, num_clusters):
    gm = GaussianMixture(n_components=num_clusters).fit(X_train)
    labels = gm.predict(X_test)
    rand = rand_score(Y_test, labels)
    
    return rand 

In [33]:
km_rand_fleece = kmeans_fabric(X_train_fleece, X_test_fleece, Y_test_fleece, 2)
gmm_rand_fleece = GMM_fabric(X_train_fleece, X_test_fleece, Y_test_fleece, 2)

km_rand_knit = kmeans_fabric(X_train_knit, X_test_knit, Y_test_knit, 2)
gmm_rand_knit = GMM_fabric(X_train_knit, X_test_knit, Y_test_knit, 2)

km_rand_linen = kmeans_fabric(X_train_linen, X_test_linen, Y_test_linen, 3)
gmm_rand_linen = GMM_fabric(X_train_linen, X_test_linen, Y_test_linen, 3)

km_rand_leather = kmeans_fabric(X_train_leather, X_test_leather, Y_test_leather, 3)
gmm_rand_leather = GMM_fabric(X_train_leather, X_test_leather, Y_test_leather, 3)

km_rand_outdoor = kmeans_fabric(X_train_outdoor, X_test_outdoor, Y_test_outdoor, 2)
gmm_rand_outdoor = GMM_fabric(X_train_outdoor, X_test_outdoor, Y_test_outdoor, 2)

km_rand_denim = kmeans_fabric(X_train_denim, X_test_denim, Y_test_denim, 2)
gmm_rand_denim = GMM_fabric(X_train_denim, X_test_denim, Y_test_denim, 2)

In [34]:
print('fleece')
print('KMeans\t:', km_rand_fleece)
print('GMM\t:', gmm_rand_fleece)

print('')
print('knit')
print('KMeans\t:', km_rand_knit)
print('GMM\t:', gmm_rand_knit)

print('')
print('linen')
print('KMeans\t:',km_rand_linen)
print('GMM\t:', gmm_rand_linen)

print('')
print('leather')
print('KMeans\t:', km_rand_leather)
print('GMM\t:', gmm_rand_leather)


print('')
print('outdoor')
print('KMeans\t:', km_rand_outdoor)
print('GMM\t:', gmm_rand_outdoor)


print('')
print('denim')
print('KMeans\t:', km_rand_denim)
print('GMM\t:', gmm_rand_denim)


fleece
KMeans	: 0.5282146160962072
GMM	: 0.4431082331174838

knit
KMeans	: 0.6516516516516516
GMM	: 0.6516516516516516

linen
KMeans	: 0.6291291291291291
GMM	: 0.6306306306306306

leather
KMeans	: 0.5781512605042017
GMM	: 0.5764705882352941

outdoor
KMeans	: 0.4974257425742574
GMM	: 0.500990099009901

denim
KMeans	: 0.543859649122807
GMM	: 0.6491228070175439
